## Motors BDC call data analysis : 2. memo 분석을 위한 데이터 전처리
- source : BMW Experience (by 원서현 주임) 
- period : 2023 1/1 ~ 9/6 (총 54,638건) 
- size : 54,638 rows x 84 columns (csv file, 35.1 MB) 
---

In [1]:
## 필수 pkg's 및 한글폰트 사용 설정 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings 
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rc('axes',unicode_minus=False)
plt.rcParams["figure.figsize"] = (8,3)
pd.set_option('display.max_columns', None)

In [11]:
## data loading 
df_temp = pd.read_csv('data/bdc_data_20230908.csv', encoding='cp949')
df_temp.shape

(54638, 84)

--- 
#### 메모 분석을 위한 데이터셋 준비 
- 접수번호와 메모1~5 컬럼만 가진 데이터 프레임 생성 
---

In [3]:
usable_columns = ['(접수)번호', 'MEMO1','MEMO2', 'MEMO3', 'MEMO4', 'MEMO5']

df = df_temp[usable_columns]
print(df.shape)
df.head()

(54638, 6)


,(접수)번호,MEMO1,MEMO2,MEMO3,MEMO4,MEMO5
0,959430,차대번호 : GA84735 보유차량 : 3GT 20d LCI 등록일자 : 2019-...,프로모션 문의 여부 : 관심시리즈:,서비스 문의 여부 : 문의 내용:,Trade-in 문의 여부 :,부재
1,959429,차대번호 : K736791 보유차량 : 320d LCI 등록일자 : 2019-01-...,프로모션 문의 여부 : 관심시리즈:,서비스 문의 여부 : 문의 내용:,Trade-in 문의 여부 :,부재
2,959428,차대번호 : BP71939 보유차량 : 520i 등록일자 : 2019-01-24 서...,프로모션 문의 여부 : 관심시리즈:,서비스 문의 여부 :,Trade-in 문의 여부 :,상담거부
3,959427,차대번호 : 0Z83228 보유차량 : New X6 xDrive30d 등록일자 : ...,프로모션 문의 여부 : 관심시리즈:,서비스 문의 여부 : 문의 내용:,Trade-in 문의 여부 :,당사직원
4,959426,차대번호 : K736629 보유차량 : 320d LCI 등록일자 : 2019-02-...,프로모션 문의 여부 : 관심시리즈:,서비스 문의 여부 : 문의 내용:,Trade-in 문의 여부 :,부재


----
#### memo 분석관련 아이디어 
- 메모 내용에 특정한 패턴이 존재한다고 보기 어려워 분석 곤란, 우선 다음과 같이 접근 
- MEMO2에서 가장 많은 케이스는 "프로모션 문의 여부"와 "관심시리즈". "프로모션 문의여부:O"인 건들만 추출한 후, 텍스트 = "프로모션 문의여부:O 관심시리즈:" 삭제 -> 관심시리즈만 추출 
- MEMO3에서 가장 많은 케이스는 "서비스 문의 여부"와 "문의내용". "서비스 문의 여부 :O"인 건들만 추출한 후, 텍스트 = "서비스 문의 여부 :O 문의 내용:" 삭제 -> 서비스 문의내용만 추출 
----

In [4]:
## MEMO 2컬럼 전처리 
key_word_1 = '프로모션 문의 여부 :O'
key_word_2 = '서비스 문의 여부 :O'

## MEMO2에 key_word_1 포함 또는 MEMO3에 key_word_2 포함된 건들만 추출
df_2 = df[df['MEMO2'].str.contains(key_word_1, na=False)]
df_3 = df[df['MEMO3'].str.contains(key_word_2, na=False)]

print(df_2.shape, df_3.shape)

(430, 6) (1507, 6)


In [5]:
## df_2의 MEMO2 컬럼에서 관심시리즈의 응답값만 추출 
df_2['관심시리즈'] = df_2['MEMO2'].str[20:]
df_2['관심시리즈']

118            7시리즈 신형
129               7시리즈
157              X1 신형
225      아직관심모델생각해놓지않음
262          X5, X7 디젤
             ...      
53910             5시리즈
53956               X3
54300          5시리즈 신형
54376               XM
54480               X4
Name: 관심시리즈, Length: 430, dtype: object

In [6]:
## df_2의 MEMO3 컬럼에서 문의내용의 응답값만 추출 
df_3['서비스문의내용'] = df_3['MEMO3'].str[19:]
df_3['서비스문의내용']

23                                            뒷브레이크 패드 점검등
40                                                 EGR리콜예약
75                                               리콜요청 AS요청
87                            열선 따뜻하면 되고, 추울때는 작동안한다하심AS요청
95                   타이어 공기압 채워도 계속 경고등 뜨고, 브레이크액 경고등 AS요청
                               ...                        
54461                                             엔진경고등 점등
54520    비상호출시스템 문제 있는거 발견돼서 센터에서 예약 잡고 연락준다 하였으나 연락없어서 문의
54537                               타이어공기압 안맞춰짐 발란스맞춤 예약요청
54580    정비불량으로 차량 다시 맡긴건데 대차차량 없다며 3주 기다리라고 했다며 불만,빠른시...
54637                                     엔진오일 및 브레이크오일 점등
Name: 서비스문의내용, Length: 1507, dtype: object

In [7]:
## df_2, df_3를 접수번호 기준으로 조인하여 최종 데이터셋 만들기 
df_4 = pd.merge(df_2, df_3, how='outer', on='(접수)번호')
print(df_4.shape)
df_4.head(1)

(1884, 13)


,(접수)번호,MEMO1_x,MEMO2_x,MEMO3_x,MEMO4_x,MEMO5_x,관심시리즈,MEMO1_y,MEMO2_y,MEMO3_y,MEMO4_y,MEMO5_y,서비스문의내용
0,959218,차대번호 : G573350 보유차량 : 3GT 20d xDrive_New Engin...,프로모션 문의 여부 :O 관심시리즈:7시리즈 신형,서비스 문의 여부 : 문의 내용:,"Trade-in 문의 여부 :O(70,000km) 트레이드인프로모션 상담요청",구매상담요청 프로모션종합상담,7시리즈 신형,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_f = df_4[['(접수)번호', '관심시리즈', '서비스문의내용']]
print(df_f.shape)
df_f.head()

(1884, 3)


,(접수)번호,관심시리즈,서비스문의내용
0,959218,7시리즈 신형,NaN
1,959207,7시리즈,NaN
2,959179,X1 신형,NaN
3,958540,아직관심모델생각해놓지않음,NaN
4,958503,"X5, X7 디젤",NaN


In [9]:
## 전처리 완료된 파일 내보내기 
df_f.to_csv('data/df_f.csv', index=False, encoding='utf-8-sig')